In [1]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/AUP-CS2091/Week-8/master/churn_train.csv?token=AFj8VRPCOLh9_gf-OjwUGWJJhbn7-Lhnks5Z_IS4wA%3D%3D')
#tdata = pd.read_csv('https://raw.githubusercontent.com/AUP-CS2091/Week-7/master/churn_test.csv?token=AFj8VUJJdGcDbMzoty9Joi26M-XTPoOtks5Z8iftwA%3D%3D')

data.head()

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,LEAVE
0,zero,31953,0,6,313378,161,0,4,unsat,little,no,STAY
1,one,36147,0,13,800586,244,0,6,unsat,little,considering,STAY
2,one,27273,230,0,305049,201,16,15,unsat,very_little,perhaps,STAY
3,zero,120070,38,33,788235,780,3,2,unsat,very_high,considering,LEAVE
4,one,29215,208,85,224784,241,21,1,very_unsat,little,never_thought,STAY


* COLLEGE : Is the customer college educated?
* INCOME	: Annual income
* OVERAGE	: Average overcharges per month
* LEFTOVER : Average % leftover minutes per month
* HOUSE : Value of dwelling (from census tract)
* HANDSET_PRICE : Cost of phone
* OVER_15MINS_CALLS_PER_MONTH : Average number of long (>15 mins) calls per month
* AVERAGE_CALL_DURATION : Average call duration
* REPORTED_SATISFACTION : Reported level of satisfaction
* REPORTED_USAGE_LEVEL : Self-reported usage level
* CONSIDERING_CHANGE_OF_PLAN : Was customer considering changing his/her plan?
* LEAVE : Class variable: whether customer left or stayed

In [2]:
data['COLLEGE'] = data['COLLEGE'].astype('category').cat.codes
data['REPORTED_SATISFACTION'] = data['REPORTED_SATISFACTION'].astype('category').cat.codes
data['REPORTED_USAGE_LEVEL'] = data['REPORTED_USAGE_LEVEL'].astype('category').cat.codes
data['CONSIDERING_CHANGE_OF_PLAN'] = data['CONSIDERING_CHANGE_OF_PLAN'].astype('category').cat.codes
data['LEAVE'] = data['LEAVE'].astype('category').cat.codes

tdata['COLLEGE'] = tdata['COLLEGE'].astype('category').cat.codes
tdata['REPORTED_SATISFACTION'] = tdata['REPORTED_SATISFACTION'].astype('category').cat.codes
tdata['REPORTED_USAGE_LEVEL'] = tdata['REPORTED_USAGE_LEVEL'].astype('category').cat.codes
tdata['CONSIDERING_CHANGE_OF_PLAN'] = tdata['CONSIDERING_CHANGE_OF_PLAN'].astype('category').cat.codes

from sklearn.tree import DecisionTreeClassifier

X = data.loc[:, data.columns != 'LEAVE']
y = data['LEAVE']

X_pred = tdata

model = DecisionTreeClassifier(max_depth=10)
model.fit(X,y)
probs = model.predict_proba(X_pred)

tdata['PLEAVE'] = probs[:,0]
tdata.head()

NameError: name 'tdata' is not defined

In [ ]:
import numpy as np
rng = np.arange(1,40,5)
rng

In [ ]:
from sklearn.metrics import accuracy_score
acc = []
X = data.loc[:, data.columns != 'LEAVE']
y = data.loc[:, 'LEAVE']
for s in rng:
    model = DecisionTreeClassifier(max_depth=s)
    model.fit(X,y)
    acc.append(accuracy_score(y,model.predict(X)))

acc

In [3]:
import matplotlib.pylab as plt
%matplotlib inline
plt.scatter(rng, acc, c='red')

NameError: name 'rng' is not defined

In [4]:
X = data.loc[:, data.columns != 'LEAVE']
y = data.loc[:, 'LEAVE']
from sklearn import model_selection
X_train,X_test,y_train,y_test = \
    model_selection.train_test_split(X,y,train_size=0.8)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
err = []
xt = X_train
yt = y_train
for s in rng:
    model = DecisionTreeClassifier(max_depth=s)
    model.fit(xt,yt)
    err.append(accuracy_score(yt,model.predict(xt)))
err

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
plt.scatter(rng, err, c='blue')

In [ ]:
rng = np.arange(1, 40, 5)
acc2 = []
for s in rng:
    model = DecisionTreeClassifier(max_depth=s)
    model.fit(X_train,y_train)
    acc2.append(accuracy_score(y_test, model.predict(X_test)))
    
import matplotlib.pylab as plt
%matplotlib inline
plt.scatter(rng, acc2, c='blue')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
model = DecisionTreeClassifier(max_depth=10)
scores = cross_val_score(model, X,y,cv=5)
mean = scores.mean()
rng = np.arange(1,40,5)
scor=[]
for s in rng:
    model = DecisionTreeClassifier(max_depth=s)
    model.fit(X,y)
    scor.append(cross_val_score(model,X,y,cv=5).mean())
    
import matplotlib.pylab as plt
%matplotlib inline
plt.scatter(rng, scor, c='red')

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(model,X,y,cv=5)
mean_values = np.mean(test_scores, axis=1)